In [1]:
import pandas as pd
import os
import logging

# Read the CSV dataset
file_path = '../datasets/movies_metadata_051424.csv'
df = pd.read_csv(file_path)

df = df.fillna('')

print(len(df))
df.head()

10638


,id,title,summary,year,certificate,runtime,runtime_mins,rating,votes,director_1,...,cast_2,cast_3,genre_1,genre_2,genre_3,sentiment_score,sentiment_reason,recommended_audience,img,metadata
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen...,2024,PG-13,2h 46m,166,8.7,360000,Denis Villeneuve,...,Zendaya,Rebecca Ferguson,Action,Adventure,Drama,3,The movie 'Dune: Part Two' showcases themes of...,This movie is recommended for viewers who appr...,https://m.media-amazon.com/images/M/MV5BN2QyZG...,Dune: Part Two | Denis Villeneuve | Timothée C...
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash i...",2024,PG-13,1h 55m,115,6.5,37000,Adam Wingard,...,Brian Tyree Henry,Dan Stevens,Action,Adventure,Fantasy,7,The movie ends on a positive note with Godzill...,This film is recommended for audiences who enj...,https://m.media-amazon.com/images/M/MV5BY2QwOG...,Godzilla x Kong: The New Empire | Adam Wingard...
2,23137904,Rebel Moon - Part Two: The Scargiver,Kora and surviving warriors prepare to defend ...,2024,PG-13,2h 2m,122,5.2,17000,Zack Snyder,...,Djimon Hounsou,Ed Skrein,Action,Adventure,Drama,7,"The synopsis highlights themes of unity, resil...",This movie is recommended for audiences who en...,https://m.media-amazon.com/images/M/MV5BYmQ2OD...,Rebel Moon - Part Two: The Scargiver | Zack Sn...
3,21692408,Kung Fu Panda 4,After Po is tapped to become the Spiritual Lea...,2024,PG,1h 34m,94,6.4,30000,Mike Mitchell,...,Awkwafina,Viola Davis,Animation,Action,Adventure,8,The synopsis portrays a heartwarming story of ...,This movie is recommended for audiences seekin...,https://m.media-amazon.com/images/M/MV5BZDY0Yz...,"Kung Fu Panda 4 | Mike Mitchell | Jack Black, ..."
4,1160419,Dune,A noble family becomes embroiled in a war for ...,2021,PG-13,2h 35m,155,8.0,851000,Denis Villeneuve,...,Rebecca Ferguson,Zendaya,Action,Adventure,Drama,6,The synopsis portrays a mix of challenges and ...,Recommended for viewers who appreciate epic ta...,https://m.media-amazon.com/images/M/MV5BMDQ0Nj...,"Dune | Denis Villeneuve | Timothée Chalamet, R..."


In [8]:
# Create a new column named 'metadata' with all the data combined
def generate_metadata(row):
    metadata = (
        f"{row['title']} | "
        f"{row['summary']}"
    )
    return metadata

df['metadata'] = df.apply(generate_metadata, axis=1)

pd.set_option('display.max_colwidth', None)
print("A sample value from column 'metadata':")
df['metadata'].head(1)

A sample value from column 'metadata':


0    Dune: Part Two | Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.
Name: metadata, dtype: object

In [2]:
from openai import OpenAI
import openai
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

# Generate the vector embeddings for selected columns
def get_embedding(text, model='text-embedding-3-small'):
    try:
        # text = text.replace('\n', ' ')
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        logging.error(f'Error generating embeddings: {e}. Found issue in the following text: {text}.')
        text = 'No data available'
        return client.embeddings.create(input = [text], model=model).data[0].embedding

tqdm.pandas()
df['metadata_vector'] = df['metadata'].progress_apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
print('Conversion to vector embeddings has been completed.')

df.head()

100%|███| 10638/10638 [29:44<00:00,  5.96it/s]

Conversion to vector embeddings has been completed.


,id,title,summary,year,certificate,runtime,runtime_mins,rating,votes,director_1,...,cast_3,genre_1,genre_2,genre_3,sentiment_score,sentiment_reason,recommended_audience,img,metadata,metadata_vector
0,15239678,Dune: Part Two,Paul Atreides unites with Chani and the Fremen...,2024,PG-13,2h 46m,166,8.7,360000,Denis Villeneuve,...,Rebecca Ferguson,Action,Adventure,Drama,3,The movie 'Dune: Part Two' showcases themes of...,This movie is recommended for viewers who appr...,https://m.media-amazon.com/images/M/MV5BN2QyZG...,Dune: Part Two | Denis Villeneuve | Timothée C...,"[-0.022296011447906494, 0.044338658452034, -0...."
1,14539740,Godzilla x Kong: The New Empire,"Two ancient titans, Godzilla and Kong, clash i...",2024,PG-13,1h 55m,115,6.5,37000,Adam Wingard,...,Dan Stevens,Action,Adventure,Fantasy,7,The movie ends on a positive note with Godzill...,This film is recommended for audiences who enj...,https://m.media-amazon.com/images/M/MV5BY2QwOG...,Godzilla x Kong: The New Empire | Adam Wingard...,"[-0.004718017764389515, 0.05989082157611847, 0..."
2,23137904,Rebel Moon - Part Two: The Scargiver,Kora and surviving warriors prepare to defend ...,2024,PG-13,2h 2m,122,5.2,17000,Zack Snyder,...,Ed Skrein,Action,Adventure,Drama,7,"The synopsis highlights themes of unity, resil...",This movie is recommended for audiences who en...,https://m.media-amazon.com/images/M/MV5BYmQ2OD...,Rebel Moon - Part Two: The Scargiver | Zack Sn...,"[0.024749718606472015, 0.075119748711586, 0.01..."
3,21692408,Kung Fu Panda 4,After Po is tapped to become the Spiritual Lea...,2024,PG,1h 34m,94,6.4,30000,Mike Mitchell,...,Viola Davis,Animation,Action,Adventure,8,The synopsis portrays a heartwarming story of ...,This movie is recommended for audiences seekin...,https://m.media-amazon.com/images/M/MV5BZDY0Yz...,"Kung Fu Panda 4 | Mike Mitchell | Jack Black, ...","[-0.019859328866004944, 0.011722062714397907, ..."
4,1160419,Dune,A noble family becomes embroiled in a war for ...,2021,PG-13,2h 35m,155,8.0,851000,Denis Villeneuve,...,Zendaya,Action,Adventure,Drama,6,The synopsis portrays a mix of challenges and ...,Recommended for viewers who appreciate epic ta...,https://m.media-amazon.com/images/M/MV5BMDQ0Nj...,"Dune | Denis Villeneuve | Timothée Chalamet, R...","[-0.030934041365981102, 0.032390281558036804, ..."


In [3]:
df.to_csv('movies_embedding_051424.csv', index=False)